In [1]:
import os
from get_data import load_data
from utils import calculate_metrics, load_model, make_predictions
from tqdm import tqdm
import numpy as np

MODELS_DIR = "./models"
imb_rate = 0.01  # Imbalance rate
min_class = [1]  # Minority classes, must be same as trained model
maj_class = [0]  # Majority classes, must be same as trained model
datasource = "credit"  # The dataset to be selected

In [2]:
# Remove classes ∉ {min_class, maj_class}, imbalance the dataset
# Make sure the same seed is used as during training to ensure no data contamination
X_train, y_train, X_test, y_test, X_val, y_val = load_data(datasource, imb_rate, min_class, maj_class)  # Load all data

Imbalance ratio `p`:
	dataset:    n=492, p=0.001730
	train:      n=295, p=0.001729
	test:       n=98, p=0.001723
	validation: n=99, p=0.001741


In [3]:
all_stats = {"Gmean": [], "Fdot5": [], "F1": [], "F2": [], "TP": [], "TN": [], "FP": [], "FN": []}

for fp_model in tqdm(os.listdir(MODELS_DIR)):
    model = load_model(f"{MODELS_DIR}/{fp_model}")
    y_pred = make_predictions(model, X_test)
    stats = calculate_metrics(y_test, y_pred)
    
    for k, v in stats.items():
        all_stats[k].append(v)

100%|██████████| 31/31 [01:34<00:00,  3.06s/it]


In [4]:
for k in all_stats.keys():
    print(f"{k}: {np.mean(all_stats[k]):.6f}", end=" ")

Gmean: 0.497058 Fdot5: 0.011255 F1: 0.017559 F2: 0.040030 TP: 71.516129 TN: 31701.774194 FP: 25162.225806 FN: 26.483871 

In [5]:
maxF1 = max(all_stats.get("F1"))
maxF1_index = all_stats.get("F1").index(maxF1)
f"Best scoring model is: {os.listdir(MODELS_DIR)[maxF1_index]} with F1 score of {maxF1:.6f}"

'Best scoring model is: 20200928_FN16_FP35.h5 with F1 score of 0.045603'